1. Install required NuGets

In [1]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1

2. Add global usings

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using System.IO;
using System.Text;

3. Declare your data models. Input and Output

In [1]:
public class RealEstateInput
{
    [LoadColumn(0)]
    public float Id { get; set; }

    [LoadColumn(1)]
    public float TransactionDate { get; set; }

    [LoadColumn(2)]
    public float HouseAge { get; set; }

    [LoadColumn(3)]
    public float NearestMRT { get; set; }

    [LoadColumn(4)]
    public float NumberOfStores { get; set; }

    [LoadColumn(5)]
    public float Latitude { get; set; }

    [LoadColumn(6)]
    public float Longitude { get; set; }

    [LoadColumn(7)]
    public float HousePrice { get; set; }
}

public class RealEstateOutput
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

4. Load the data set and split into training and test sets

In [1]:
var mlContext = new MLContext();

var data = mlContext.Data.LoadFromTextFile<RealEstateInput>("../../../ML.NET.Demo/Assets/Real estate.csv", hasHeader: true, separatorChar: ',');

//Usually want to take 20% of your complete data set to train. Of course you can play with this number to aid in accuracy
var dataSplit = mlContext.Data.TrainTestSplit(data, testFraction: 0.2);

5. You can visually display dataset tabicularly 

In [1]:
dataSplit.TrainSet.ToTabularDataResource().Display();

Id,TransactionDate,HouseAge,NearestMRT,NumberOfStores,Latitude,Longitude,HousePrice
1,2012.917,32,84.87882,10,24.98298,121.54024,37.9
2,2012.917,19.5,306.5947,9,24.98034,121.53951,42.2
3,2013.583,13.3,561.9845,5,24.98746,121.54391,47.3
4,2013.5,13.3,561.9845,5,24.98746,121.54391,54.8
5,2012.833,5,390.5684,5,24.97937,121.54245,43.1
8,2013.417,20.3,287.6025,6,24.98042,121.54228,46.7
9,2013.5,31.7,5512.038,1,24.95095,121.48458,18.8
10,2013.417,17.9,1783.18,3,24.96731,121.51486,22.1
11,2013.083,34.8,405.2134,1,24.97349,121.53372,41.4
12,2013.333,6.3,90.45606,9,24.97433,121.5431,58.1


6. Train our model. (Feature Engineering)

In [1]:
var model = mlContext.Regression.Trainers.Sdca(labelColumnName: "Label", featureColumnName: "Features");

var pipeline = mlContext.Transforms.Concatenate("Features", new[] {
                                                        nameof(RealEstateInput.NearestMRT),
                                                        nameof(RealEstateInput.Latitude),
                                                        nameof(RealEstateInput.TransactionDate),
                                                        nameof(RealEstateInput.Longitude),
                                                        nameof(RealEstateInput.NumberOfStores),
                                                        nameof(RealEstateInput.HouseAge)})
                        .Append(mlContext.Transforms.CopyColumns("Label", nameof(RealEstateInput.HousePrice)))
                        .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
						.Append(model);

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

7. Evaluate our model

In [1]:
var testSet = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.Regression.Evaluate(testSet);

metrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
5.866253241514548,59.99777686560458,7.745823188377371,59.9977772785679,0.6429988800628035


8. Save our model

In [1]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./LinearRegressionModel.mdl");

9. Load our saved model and test it

In [1]:
var newSample = new RealEstateInput
{
    Id = 1f,
    TransactionDate = 2012.917f,
    HouseAge = 32f,
    NearestMRT = 84.87882f,
    NumberOfStores = 10f,
    Latitude = 24.98298f,
    Longitude = 121.54024f
};

using (var stream = new FileStream("./LinearRegressionModel.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var model = mlContext.Model.Load(stream, out var _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<RealEstateInput, RealEstateOutput>(model);
    
    var results = predictionEngine.Predict(newSample);

    Console.WriteLine(results.Price);
}

47.377594
